In [2]:
import pandas as pd
import pprint

pp = pprint.PrettyPrinter(indent=2, compact=True)

In [9]:
INDENT = ""*2
OUTPUT = "../lib/data/topics.dart"

In [10]:
df = pd.read_csv("../data/topics.csv").fillna('')

In [11]:
topic_groups = []

cur_topic_group = {}
cur_topic = {}
cur_item = {}
for row in df.to_dict(orient="records"):
    if row["topicGroup"]:
        cur_topic_group = {
            "id": row["topicGroup"],
            "title": row["topicGroup_title"],
            "description": row["topicGroup_desc"],
            "topics": [],
            "tags": [],
        }
        topic_groups.append(cur_topic_group)
    
    if row["topic"]:
        cur_topic = {
            "id": f'{cur_topic_group["id"]}_{row["topic"]}',
            "title": row["topic_title"],
            "description": row["topicGroup_desc"],
            "items": [],
            "tags": [],
        }
        if cur_topic_group:
            cur_topic_group["topics"].append(cur_topic)
        
        
    if row["item"]:
        cur_item = {
            "id": f'{cur_topic["id"]}_{row["item"]}',
            "title": row["item_title"],
            "body": row["item_body"],
            "order_index": row["item_orderIndex"],
        }
        if cur_topic: 
            cur_topic["items"].append(cur_item)

In [13]:
def item_to_string(d: dict) -> str:
    args = [
        f'id: "{d["id"]}"',
        f'body: "{d["body"]}"',
    ]
    if d["title"] != "":
        args.append(f'title: "{d["title"]}"')
    
    return "Item(" + ", ".join(args) + "),"


def topic_to_string(d: dict) -> str:
    args = [
        f'id: "{d["id"]}"',
        f'title: "{d["title"]}"',
    ]
    if d["description"] != "":
        args.append(f'description: "{d["description"]}"')
        
    items = [INDENT*3 + item_to_string(item) for item in d["items"]]
    items_str = "\n".join(items)
    args.append(f'items: [\n{items_str}\n{INDENT*2}]')
        
    return "Topic(" + ", ".join(args) + "),"


def topic_group_to_string(d: dict) -> str:
    args = [
        f'id: "{d["id"]}"',
        f'title: "{d["title"]}"',
    ]
    if d["description"] != "":
        args.append(f'description: "{d["description"]}"')
        
    topics = [INDENT*2 + topic_to_string(topic) for topic in d["topics"]]
    topics_str = "\n".join(topics)
    args.append(f'topics: [\n{topics_str}\n{INDENT}]')
        
    return "TopicGroup(" + ", ".join(args) + "),"
        
    
def data_to_dart(d: list) -> str:
    preamble = "import 'package:date_topics/domain/domain.dart';\n\n"
    
    topic_groups = [INDENT + topic_group_to_string(tg)+"\n" for tg in d]
    
    return preamble + "final List<TopicGroup> topicGroupsData = [\n" + "".join(topic_groups) + "];"
     

with open(OUTPUT, "w") as f:
    f.write(data_to_dart(topic_groups))
    
print("generation done")


generation done
